In [21]:
import datetime
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, HiveContext, SQLContext


from pyspark.sql.functions import (
    col,
    split
)

spark_context =SparkContext.getOrCreate()

spark = SparkSession.builder \
    .master("yarn") \
    .appName("AspectDetector") \
    .enableHiveSupport() \
    .getOrCreate()

sqlContext = SQLContext(spark)

# Edit S3 Paths here:
s3_bucket = "data-team"
s3_subpath = "yifanliu"
s3_fullpath = "s3://" + s3_bucket + "/" + s3_subpath + "/"

print(s3_fullpath)

#change parameter here
today = datetime.datetime.now()
schema='yifanliu'
temp_event_funnel='temp_rider_onboarding_funnel_events'
output_path=s3_fullpath+temp_event_funnel
#visit_temp='tablename=temp_rider_onboarding_funnel_events'
lookback=1 #read data lookback window

s3://data-team/yifanliu/


In [33]:
def get_funnel_events(spark, date, lookback, event_name, other_condition=''):
    hql = """
        SELECT *
        FROM yifanliu.temp_rider_onboarding_funnel_events a
        WHERE a.ds >= CAST(date_add('{date}',-{lookback}) as STRING) and event_name='{event_name}' {other}
    """.format(date=date, lookback=lookback, event_name=event_name, other=other_condition)
    print(hql)

    return spark.sql(hql)

In [34]:
benefits_redeemed=get_funnel_events(spark,today,lookback,'benefit_redeemed')
invites_sent=get_funnel_events(spark,today,lookback,'invites_sent')
invite_clicked=get_funnel_events(spark,today,lookback,'invite_clicked')


        SELECT *
        FROM yifanliu.temp_rider_onboarding_funnel_events a
        WHERE a.ds >= CAST(date_add('2021-02-26 23:21:14.909596',-1) as STRING) and event_name='benefit_redeemed' 
    

        SELECT *
        FROM yifanliu.temp_rider_onboarding_funnel_events a
        WHERE a.ds >= CAST(date_add('2021-02-26 23:21:14.909596',-1) as STRING) and event_name='invites_sent' 
    

        SELECT *
        FROM yifanliu.temp_rider_onboarding_funnel_events a
        WHERE a.ds >= CAST(date_add('2021-02-26 23:21:14.909596',-1) as STRING) and event_name='invite_clicked' 
    


In [35]:
benefits_redeemed.show()

+--------------------+-------------------+------------------------+----------------+-----------------------+-------------+----------+----------------+
|         occurred_at|       user_lyft_id|ride_program_customer_id|     coupon_code|ride_program_benefit_id|       source|        ds|      event_name|
+--------------------+-------------------+------------------------+----------------+-----------------------+-------------+----------+----------------+
|2021-02-25 16:46:...|1511507998969326310|     1511689347926187118|7OZ31BVZK96HND0K|    1511689347926187124|auto_redeemed|2021-02-25|benefit_redeemed|
|2021-02-25 16:35:...|1509239447715595054|     1511686530171007536|7OZ31BVZK96HND0K|    1511686530171007542|auto_redeemed|2021-02-25|benefit_redeemed|
|2021-02-25 16:53:...|1079849853651104784|     1511691006048692648|BTJBQHSHQU8U9O7S|    1511691006048692654|auto_redeemed|2021-02-25|benefit_redeemed|
|2021-02-25 16:52:...|1505356169792662048|     1506160530432888162|PYCAWUD2RP3GOR5Q|    151169

In [36]:
welcome_xp_seen=get_funnel_events(spark,today,lookback,'ux_displayed',"and source in ('welcome_modal','welcome_v2_modal')")
click_see_details=get_funnel_events(spark,today,lookback,'ux_tapped',"and source in ('program_details_prompt','welcome_v2_modal_continue_button')")
welcome_xp_click_done=get_funnel_events(spark,today,lookback,'ux_tapped',"and source in ('welcome_modal_done_button', 'program_details_v2_done_button')")


        SELECT *
        FROM yifanliu.temp_rider_onboarding_funnel_events a
        WHERE a.ds >= CAST(date_add('2021-02-26 23:21:14.909596',-1) as STRING) and event_name='ux_displayed' and source in ('welcome_modal','welcome_v2_modal')
    

        SELECT *
        FROM yifanliu.temp_rider_onboarding_funnel_events a
        WHERE a.ds >= CAST(date_add('2021-02-26 23:21:14.909596',-1) as STRING) and event_name='ux_tapped' and source in ('program_details_prompt','welcome_v2_modal_continue_button')
    

        SELECT *
        FROM yifanliu.temp_rider_onboarding_funnel_events a
        WHERE a.ds >= CAST(date_add('2021-02-26 23:21:14.909596',-1) as STRING) and event_name='ux_tapped' and source in ('welcome_modal_done_button', 'program_details_v2_done_button')
    


In [41]:
benefits_redeemed.createOrReplaceTempView('benefits_redeemed')
invites_sent.createOrReplaceTempView('invites_sent')
invite_clicked.createOrReplaceTempView('invite_clicked')

welcome_xp_seen.createOrReplaceTempView('welcome_xp_seen')
click_see_details.createOrReplaceTempView('click_see_details')
welcome_xp_click_done.createOrReplaceTempView('welcome_xp_click_done')


In [39]:
combine_invite_clicked = '''
select br.occurred_at benefit_redeemed_at,
       br.user_lyft_id as user_lyft_id_redemption, 
       br.ride_program_customer_id, 
       br.coupon_code, 
       br.ride_program_benefit_id,  
       br.source claim_source,
       isa.occurred_at invites_sent_at,
       isa.user_lyft_id user_lyft_id_invite_sent,
       ic.occurred_at as invite_clicked_at,
       ic.user_lyft_id as user_lyft_id_invite_click,
       ic.source as code
  from benefits_redeemed br
  left join invites_sent isa
    on br.user_lyft_id = isa.user_lyft_id
   and date_trunc('day', isa.occurred_at) >= date_trunc('day', br.occurred_at) -- track invites sent after benefit redemption
   and date_trunc('day', isa.occurred_at) < date_trunc('day', br.occurred_at) + interval '2' day -- track invites sent within 2 days of redemption
  left join invite_clicked ic
    on br.user_lyft_id = ic.user_lyft_id
   and date_trunc('day', ic.occurred_at) >= date_trunc('day', br.occurred_at)
   and date_trunc('day', ic.occurred_at) < date_trunc('day', br.occurred_at) + interval '30' day -- track clicks within 30 days of sent
where br.ds >= CAST(date_add('{date}',-{lookback}) as STRING) and br.ds <= cast('{date}' as string)
'''.format(date=today, lookback=30)

#print(combine_invite_clicked)
combine_invite_clicked = spark.sql(combine_invite_clicked)

combine_invite_clicked.createOrReplaceTempView('combine_invite_clicked')

#combine_invite_clicked.show()


select br.occurred_at benefit_redeemed_at,
       br.user_lyft_id as user_lyft_id_redemption, 
       br.ride_program_customer_id, 
       br.coupon_code, 
       br.ride_program_benefit_id,  
       br.source claim_source,
       isa.occurred_at invites_sent_at,
       isa.user_lyft_id user_lyft_id_invite_sent,
       ic.occurred_at as invite_clicked_at,
       ic.user_lyft_id as user_lyft_id_invite_click,
       ic.source as code
  from benefits_redeemed br
  left join invites_sent isa
    on br.user_lyft_id = isa.user_lyft_id
   and date_trunc('day', isa.occurred_at) >= date_trunc('day', br.occurred_at) -- track invites sent after benefit redemption
   and date_trunc('day', isa.occurred_at) < date_trunc('day', br.occurred_at) + interval '2' day -- track invites sent within 2 days of redemption
  left join invite_clicked ic
    on br.user_lyft_id = ic.user_lyft_id
   and date_trunc('day', ic.occurred_at) >= date_trunc('day', br.occurred_at)
   and date_trunc('day', ic.occurred_at) <

In [43]:
prep = """
select date_trunc('month', benefit_redeemed_at) as benefit_redeemed_at,
       count(distinct ic.user_lyft_id_redemption) as num_ppl_redeemed_benefit,
       count(distinct ic.user_lyft_id_invite_sent) as num_ppl_receive_sms_push,
       count(distinct ic.user_lyft_id_invite_click) as num_ppl_click_invite_mobile,
       count(distinct ws.user_lyft_id) as num_ppl_see_welcome_screen,
       count(distinct csd.user_lyft_id) as num_ppl_see_details,
       count(distinct wd.user_lyft_id) as num_ppl_click_done
  from combine_invite_clicked ic
  left join welcome_xp_seen ws
    on ic.user_lyft_id_redemption = ws.user_lyft_id
   and date_trunc('day', ws.occurred_at) = date_trunc('day', invites_sent_at)
   and date_trunc('day', ws.occurred_at) < date_trunc('day', invites_sent_at) + interval '30' day -- track welcome screen view within 30 days of sent
  left join click_see_details csd
    on ic.user_lyft_id_redemption = csd.user_lyft_id
   and date_trunc('day', csd.occurred_at) = date_trunc('day', invites_sent_at)
   and date_trunc('day', csd.occurred_at) < date_trunc('day', invites_sent_at) + interval '30' day -- track see details click within 30 days of sent
  left join welcome_xp_click_done wd
    on ic.user_lyft_id_redemption = wd.user_lyft_id
   and date_trunc('day', wd.occurred_at) = date_trunc('day', invites_sent_at)
   and date_trunc('day', wd.occurred_at) < date_trunc('day', invites_sent_at) + interval '30' day -- track welcome screen done click within 30 days of sent
 group by 1

"""

print(prep)
prep = spark.sql(prep)
prep.show()



select date_trunc('month', benefit_redeemed_at) as benefit_redeemed_at,
       count(distinct ic.user_lyft_id_redemption) as num_ppl_redeemed_benefit,
       count(distinct ic.user_lyft_id_invite_sent) as num_ppl_receive_sms_push,
       count(distinct ic.user_lyft_id_invite_click) as num_ppl_click_invite_mobile,
       count(distinct ws.user_lyft_id) as num_ppl_see_welcome_screen,
       count(distinct csd.user_lyft_id) as num_ppl_see_details,
       count(distinct wd.user_lyft_id) as num_ppl_click_done
  from combine_invite_clicked ic
  left join welcome_xp_seen ws
    on ic.user_lyft_id_redemption = ws.user_lyft_id
   and date_trunc('day', ws.occurred_at) = date_trunc('day', invites_sent_at)
   and date_trunc('day', ws.occurred_at) < date_trunc('day', invites_sent_at) + interval '30' day -- track welcome screen view within 30 days of sent
  left join click_see_details csd
    on ic.user_lyft_id_redemption = csd.user_lyft_id
   and date_trunc('day', csd.occurred_at) = date_trunc('da